In [ ]:
import pandas as pd

In [ ]:
from google.colab import userdata

'ij5SBGB3u4e07ivGijQLzUX0H9YKp0KR9Bwdprf1'

In [ ]:
### Set api key:
api_key = userdata.get("ALPACA_API_KEY")
secret_key = userdata.get("ALPACA_SECRET_KEY")

In [ ]:
import alpaca_trade_api as alpaca
import requests
import pandas
import datetime

# Base function for querying the Alpaca.Markets API
def query_alpaca_api(API_KEY: str, API_SECRET: str, url: str, params: dict) -> dict:
    """Base function for querying the Alpaca.Markets API

    Args:
        API_KEY (str): _description_
        API_SECRET (str): _description_
        url (str): the URL to query
        params (dict): the parameters to pass to the API

    Raises:
        ValueError: _description_
        ValueError: _description_
        exception: _description_
        ValueError: _description_

    Returns:
        dict: _description_
    """ 
    # Check that the API Key and Secret are not None
    if API_KEY is None:
        raise ValueError("The API Key is not set.")
    if API_SECRET is None:
        raise ValueError("The API Secret is not set.")

    # Set the header information
    headers = {
        'accept': 'application/json',
        'APCA-API-KEY-ID': API_KEY,
        'APCA-API-SECRET-KEY': API_SECRET
    }

    try:
        # Get the response from the API endpoint
        response = requests.get(url, headers=headers, params=params)
    except Exception as exception:
        print(f"An exception occurred when querying the URL {url} with the parameters {params}: {exception}")
        raise exception
    # Get the response code
    response_code = response.status_code
    # If the response code is 403, print that the API key and or secret are incorrect
    if response_code == 403:
        print("The API key and or secret are incorrect.")
        raise ValueError("The API key and or secret are incorrect.")
    # Convert the response to JSON
    json_response = response.json()

    # Return the JSON response
    return json_response

# Function to retrieve historical candlestick data from Alpaca.Markets
def get_historic_bars(API_KEY: str, API_SECRET: str, symbols: list, timeframe: str, limit: int, start_date: datetime, end_date: datetime) -> pandas.DataFrame:
    """Function to retrieve historical candlestick data from Alpaca.Markets

    Args:
        API_KEY (str): _description_
        API_SECRET (str): _description_
        symbols (list): the symbols to retrieve the historical data for
        timeframe (str): the timeframe to retrieve the historical data for
        limit (int): the number of bars to retrieve
        start_date (datetime): the start date for the historical data
        end_date (datetime): the end date for the historical data

    Raises:
        ValueError: _description_
        ValueError: _description_
        ValueError: _description_
        exception: _description_

    Returns:
        pandas.DataFrame: _description_
    """

    # Check that the start_date and end_date are datetime objects
    if not isinstance(start_date, datetime.datetime):
        print("The start_date must be a datetime object.")
        raise ValueError("The start_date must be a datetime object.")
    if not isinstance(end_date, datetime.datetime):
        raise ValueError("The end_date must be a datetime object.")

    # Check that the end date is not in the future
    if end_date > datetime.datetime.now():
        print("The end date is in the future. Setting the end date to now.")
        end_date = datetime.datetime.now()

    # Check that the start date is not after the end date
    if start_date > end_date:
        raise ValueError("The start date cannot be after the end date.")

    # Convert the symbols list to a comma-separated string
    symbols_joined = ",".join(symbols)

    # Set the start and end dates to the correct format - they should only include days
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    # Create the params dictionary
    params = {
        "symbols": symbols_joined,
        "timeframe": timeframe,
        "limit": limit,
        "start": start_date,
        "end": end_date,
        "adjustment": "raw",
        "feed": "iex",
        "sort": "asc"
    }

    # Set the API endpoint
    url = f"https://data.alpaca.markets/v2/stocks/bars"
    # Send to the base function to query the API
    try:
        json_response = query_alpaca_api(API_KEY, API_SECRET, url, params)
    except Exception as exception:
        print(f"An exception occurred in the function get_historic_bars() with the parameters {params}: {exception}")
        raise exception

    bars_df = pd.DataFrame()

    for symbol in symbols:
        symbol_bars = json_response["bars"][symbol]

        symbol_bars_df = pd.DataFrame(symbol_bars)

        symbol_bars_df["symbol"] = symbol

        # Rename the columns for clarity
        symbol_bars_df = symbol_bars_df.rename(
            columns={
                "o": "candle_open",
                "h": "candle_high",
                "l": "candle_low",
                "c": "candle_close",
                "v": "candle_volume",
                "t": "candle_timestamp",
                "vw": "vwap"
            }
        )

        # Convert the timestamp to a pandas datetime object
        symbol_bars_df["candle_timestamp"] = pd.to_datetime(symbol_bars_df["candle_timestamp"])

        bars_df = pd.concat([bars_df, symbol_bars_df])

    # Return the historical bars as a DataFrame
    return bars_df

In [ ]:
import datetime

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=7)

# Define symbols and other parameters
symbols = ["AAPL"]  # Example symbols
timeframe = "1Min"  # Minute data
limit = 10000 # The limit maximum is 10000

# Call the function with these parameters
get_historic_bars(api_key, secret_key, symbols, timeframe, limit, start_date, end_date)

,candle_close,candle_high,candle_low,n,candle_open,candle_timestamp,candle_volume,vwap,symbol
0,252.760,252.970,251.970,273,252.085,2024-12-30 14:30:00+00:00,18009,252.608829,AAPL
1,252.070,252.805,252.070,86,252.720,2024-12-30 14:31:00+00:00,6875,252.522581,AAPL
2,251.665,252.190,251.590,97,252.170,2024-12-30 14:32:00+00:00,8141,251.922302,AAPL
3,251.570,252.150,251.530,45,251.645,2024-12-30 14:33:00+00:00,9027,252.006327,AAPL
4,251.370,251.720,251.370,37,251.700,2024-12-30 14:34:00+00:00,2416,251.568158,AAPL
...,...,...,...,...,...,...,...,...,...
1875,245.080,245.100,245.050,23,245.050,2025-01-06 19:50:00+00:00,1693,245.067500,AAPL
1876,244.940,244.980,244.940,27,244.980,2025-01-06 19:51:00+00:00,1651,244.973039,AAPL
1877,244.870,244.930,244.870,13,244.930,2025-01-06 19:52:00+00:00,662,244.905000,AAPL
1878,244.925,244.960,244.925,11,244.960,2025-01-06 19:53:00+00:00,557,244.948333,AAPL


In [ ]:
import requests


def get_options_data(symbol_list):
    """
    Get trade info for a given option symbol.

    Args:
      symbol (str): Underlying option symbol we are extracting data for.

    Returns:
      pd.DataFrame: DataFrame containing the following latest trade details:
        - Greeks (delta, gamma, rho, theta, vega)
        - Implied volatility
        - Latest Quote (ask/bid price, ask/bid sell, timestamp)
        - Latest Trade (price, size, timestamp, exchange)
    """

    complete_df = pd.DataFrame()

    for symbol in symbol_list:
      url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{symbol}?feed=opra&limit=100"

      headers = {
      "accept": "application/json",
      "APCA-API-KEY-ID": api_key,
      "APCA-API-SECRET-KEY": secret_key
     }

      response = requests.get(url, headers=headers)
      if response.status_code == 200:
          data = response.json()
      else:
        print(f"error: {response.status_code}")
        return None

      if "snapshots" in data:
        snapshots = data["snapshots"]
        option_data = []

        for option, contract in snapshots.items():
            greeks = contract.get("greeks", {})
            latestQuote = contract.get("latestQuote", {})
            latestTrade = contract.get("latestTrade", {})

            option_data.append({
                "symbol": symbol,
                "option": option,
                "delta": greeks.get("delta", None),
                "gamma": greeks.get("gamma", None),
                "rho": greeks.get("rho", None),
                "theta": greeks.get("theta", None),
                "vega": greeks.get("vega", None),
                "impliedVolatility": contract.get("impliedVolatility", None),
                "askPrice": latestQuote.get("ap", None),
                "askSize": latestQuote.get("as", None),
                "bidPrice": latestQuote.get("bp", None),
                "bidSize": latestQuote.get("bs", None),
                "quoteTimestamp": latestQuote.get("t", None),
                "tradePrice": latestTrade.get("p", None),
                "tradeSize": latestTrade.get("s", None),
                "tradeTimestamp": latestTrade.get("t", None)
            })

            if option_data:
              temp_df = pd.DataFrame(option_data)
              temp_df = temp_df.dropna(axis=1, how='all')
              complete_df = pd.concat([complete_df, temp_df], ignore_index=True)

              # convert timestamp to pandas datetime object
              complete_df['quoteTimestamp'] = pd.to_datetime(complete_df['quoteTimestamp'])
              complete_df['tradeTimestamp'] = pd.to_datetime(complete_df['tradeTimestamp'])

        return complete_df


In [ ]:
# Test get_options_data function

# Test with 3 Tickers
symbols = ['AAPL', 'GOOGL', 'TSLA']
symbols_df = get_options_data(symbols)
symbols_df.to_csv("optionsTestData.csv", index=False)

# Test with S&P 500 Tickers
df = pd.read_csv("constituents.csv")
SP500_ticker_list = df['Symbol']
SP500_df = get_options_data(SP500_ticker_list)
SP500_df.to_csv("SP500_options.csv", index=False)
